In [1]:
from carbon import CarbonSimulatorUI
from carbon.simulators.carbon_simulator import (
    __version__ as uiversion,
    __date__ as uidate,
)
from jupyformat import *
import pandas as pd
import numpy as np

In [2]:
print("[carbon_simulator] version", uiversion, uidate)
jp()

[carbon_simulator] version 1.0 21/Nov/2022


# Carbon Simulation - Test 36 - Liquidity Approximation

In [3]:
Sim = CarbonSimulatorUI(verbose=False, raiseonerror=False, pair="ETHUSDC")
Sim

CarbonSimulatorUI(<0 orders, 0 trades>, pair='ETHUSDC')

In [4]:
Sim.add_order("ETH", 10, 2000, 3000)
Sim.add_order("ETH", 10, 2500, 2500)
Sim.add_order("USDC", 10*1250, 1500, 1000)
Sim.add_order("USDC", 10*1250, 1250, 1250)
Sim

CarbonSimulatorUI(<4 orders, 0 trades>, pair='ETHUSDC')

In [5]:
Sim.state()["orders"]

,id,pair,tkn,y_int,y,y_unit,p_start,p_end,p_marg,p_unit,lid
0,0,ETHUSDC,ETH,10.0,10.0,ETH,2000.0,3000.0,2000.0,USDC per ETH,0
1,1,ETHUSDC,ETH,10.0,10.0,ETH,2500.0,2500.0,2500.0,USDC per ETH,1
2,2,ETHUSDC,USDC,12500.0,12500.0,USDC,1500.0,1000.0,1500.0,USDC per ETH,2
3,3,ETHUSDC,USDC,12500.0,12500.0,USDC,1250.0,1250.0,1250.0,USDC per ETH,3


In [6]:
Sim.state().keys()

dict_keys(['orders', 'orderuis', 'trades'])

In [7]:
Sim.state()["orderuis"]

{0: CarbonOrderUI(pair=CarbonPair(tknb='ETH', tknq='USDC'), tkn='ETH', B=0.018257418583505537, S=0.004103261191492361, yint=10.0, y=10.0),
 1: CarbonOrderUI(pair=CarbonPair(tknb='ETH', tknq='USDC'), tkn='ETH', B=0.02, S=0.0, yint=10.0, y=10.0),
 2: CarbonOrderUI(pair=CarbonPair(tknb='ETH', tknq='USDC'), tkn='USDC', B=31.622776601683793, S=7.107056860390376, yint=12500.0, y=12500.0),
 3: CarbonOrderUI(pair=CarbonPair(tknb='ETH', tknq='USDC'), tkn='USDC', B=35.35533905932738, S=0.0, yint=12500.0, y=12500.0)}

In [8]:
help(Sim.state()["orderuis"][0].liquidity_approx)

Help on method liquidity_approx in module carbon.carbon_order_ui:

liquidity_approx(price1, price2, tkn=None, asperc=False) method of carbon.carbon_order_ui.CarbonOrderUI instance
    returns the approximate liquidity between start and end, in tkn
    
    :price1/2:      the start and end price of the range (in any order; quoted in price convention of pair)
    :tkn:           the token in which the liquidity is quoted (if None: base token)
    :asperc:        if True, return percentage total liquidity rather than tkn number; default is False
    :returns:       the liquidity in [price1, price2], quoted in tkn (or percent)



## Order 0 (ETH, range)

In [9]:
oui = Sim.state()["orderuis"][0]
oui.pmin, oui.pmax, oui.total_liquidity

(2000.0, 3000.0, (10.0, 'ETH'))

In [10]:
oui.liquidity_approx(500, 600, "ETH", asperc=True)

0

In [11]:
oui.liquidity_approx(3000, 3020, "ETH", asperc=True)

0

In [12]:
oui.liquidity_approx(2000, 3000, "ETH", asperc=True)

1.0

In [13]:
oui.liquidity_approx(3000, 2000, "ETH", asperc=True)

1.0

In [14]:
oui.liquidity_approx(2000, 2500, "ETH", asperc=True)

0.5

In [15]:
oui.liquidity_approx(2500, 3000, "ETH", asperc=True)

0.5

In [16]:
oui.liquidity_approx(2250, 2750, "ETH", asperc=True)

0.5

In [17]:
oui.liquidity_approx(2000, 3000, "ETH", asperc=False)

10.0

In [18]:
oui.liquidity_approx(2000, 2500, "ETH", asperc=False)

5.0

In [19]:
oui.liquidity_approx(2000, 2500, asperc=False)

[liquidity_approx] token set to ETH


5.0

In [20]:
oui.liquidity_approx(2000, 3000, "USDC", asperc=False), sqrt(2000*3000)

(24494.89742783178, 2449.489742783178)

In [21]:
oui.liquidity_approx(2250, 2750, "USDC", asperc=False), sqrt(2250*2750)*0.5

(12437.34296383275, 1243.734296383275)

## Order 1 (ETH, point)

In [22]:
oui = Sim.state()["orderuis"][1]
oui.pmin, oui.pmax, oui.total_liquidity

(2500.0, 2500.0, (10.0, 'ETH'))

In [23]:
oui.liquidity_approx(2000, 2499, "ETH", asperc=True)

0

In [24]:
oui.liquidity_approx(2501, 3020, "ETH", asperc=True)

0

In [25]:
oui.liquidity_approx(2499, 2501, "ETH", asperc=True)

1.0

In [26]:
oui.liquidity_approx(2500, 2501, "ETH", asperc=True)

1.0

In [27]:
oui.liquidity_approx(2499, 2500, "ETH", asperc=True)

0

In [28]:
oui.liquidity_approx(2500, 2500, "ETH", asperc=True)

1.0

In [29]:
oui.liquidity_approx(2499, 2501, "ETH", asperc=False)

10.0

In [30]:
oui.liquidity_approx(2499, 2501, "USDC", asperc=False)

25000.0

In [31]:
oui.liquidity_approx(2500, 2501, "USDC", asperc=False)

25000.0

In [32]:
oui.liquidity_approx(2500, 2500, "USDC", asperc=False)

25000.0

In [33]:
oui.liquidity_approx(2000, 3000, "USDC", asperc=False)

25000.0

## Order 2 (USDC, range)

In [34]:
oui = Sim.state()["orderuis"][2]
oui.pmin, oui.pmax, oui.total_liquidity

(1000.0, 1500.0, (12500.0, 'USDC'))

In [35]:
oui.liquidity_approx(500, 600, "USDC", asperc=False)

0

In [36]:
oui.liquidity_approx(1600, 2000, "USDC", asperc=False)

0

In [37]:
oui.liquidity_approx(1000, 1500, "USDC", asperc=False)

12500.0

In [38]:
oui.liquidity_approx(1000, 1100, "USDC", asperc=False)

2500.0

In [39]:
oui.liquidity_approx(1400, 1500, "USDC", asperc=False)

2500.0

In [40]:
oui.liquidity_approx(1000, 1100, "ETH", asperc=False), 2500/sqrt(1000*1100)

(2.3836564731139807, 2.3836564731139807)

In [41]:
oui.liquidity_approx(1400, 1500, "ETH", asperc=False), 2500/sqrt(1400*1500)

(1.7251638983558857, 1.7251638983558855)

## Order 3 (USDC, point)

In [42]:
oui = Sim.state()["orderuis"][3]
oui.pmin, oui.pmax, oui.total_liquidity

(1250.0, 1250.0, (12500.0, 'USDC'))

In [43]:
oui.liquidity_approx(1249, 1251, "USDC", asperc=False)

12500.0

In [44]:
oui.liquidity_approx(1000, 2000, "USDC", asperc=False)

12500.0

In [45]:
oui.liquidity_approx(1249, 1250, "USDC", asperc=False)

0

In [46]:
oui.liquidity_approx(1250, 1251, "USDC", asperc=False)

12500.0

In [47]:
oui.liquidity_approx(1249, 1251, "ETH", asperc=False)

10.0

In [48]:
oui.liquidity_approx(1000, 2000, "ETH", asperc=False)

10.0

In [49]:
oui.liquidity_approx(1249, 1250, "ETH", asperc=False)

0

In [50]:
oui.liquidity_approx(1250, 1251, "ETH", asperc=False)

10.0